In [11]:
import numpy as np
import pandas as pd

In [12]:
def get_midpoints(data, start, end, step):
    return np.arange(start, end, step)

def check_boolean_logic(data, midpoints):
    print( f"{'Time':<12} {'/c0 Y':<8} {'/p0 Y':<8} {'/g0 Y':<8}{'/c1 Obs':<8} {'/c1 Exp':<8} {'/p1 Y':<8} {'/g1 Y':<8} {'/c2 Obs':<8} {'/c2 Exp':<8}"
           f"{'/p2 Y':<8} {'/g2 Y':<8}  {'/c3 Obs':<8} {'/c3 Exp':<8}{'/p3 Y':<8} {'/g3 Y':<8}  {'/c4 Obs':<8} {'/c4 Exp':<8}")
    print("-" * 140)
    errors = []


    for t in midpoints:
        before = data[data['/c1 X'] <= t].iloc[-1]
        after = data[data['/c1 X'] > t].iloc[0]

        interpolated = {}
        for col in data.columns:
            if ' X' not in col:  # Skip non-signal columns
                t1, t2 = before['/c1 X'], after['/c1 X']
                v1, v2 = before[col], after[col]
                interpolated[col] = v1 + (v2 - v1) * (t - t1) / (t2 - t1)

        states = {key: int(interpolated[key] > 0.5) for key in interpolated.keys()}

        c1_calc = states['/g0 Y'] or (states['/p0 Y'] and states['/c0 Y'])
        c2_calc = (
            states['/g1 Y'] or
            (states['/p1 Y'] and states['/g0 Y']) or
            (states['/p1 Y'] and states['/p0 Y'] and states['/c0 Y'])
        )
        c3_calc = (
            states['/g2 Y'] or
            (states['/p2 Y'] and states['/g1 Y']) or
            (states['/p2 Y'] and states['/p1 Y'] and states['/g0 Y']) or
            (states['/p2 Y'] and states['/p1 Y'] and states['/p0 Y'] and states['/c0 Y'])
        )

        c4_calc = (
            states['/g3 Y'] or
            (states['/p3 Y'] and states['/g2 Y']) or
            (states['/p3 Y'] and states['/p2 Y'] and states['/g1 Y']) or
            (states['/p3 Y'] and states['/p2 Y'] and states['/p1 Y'] and states['/g0 Y']) or
            (states['/p3 Y'] and states['/p2 Y'] and states['/p1 Y'] and states['/p0 Y'] and c0)
        )
        print(
            f"{t:<12.2e} {states['/c0 Y']:<8} {states['/p0 Y']:<8} {states['/g0 Y']:<8}  "
            f" {states['/c1 Y']:<8} {int(c1_calc):<8} {states['/p1 Y']:<8} {states['/g1 Y']:<8} "
            f" {states['/c2 Y']:<8} {int(c2_calc):<8}"
            f"{states['/p2 Y']:<8} {states['/g2 Y']:<8}{states['/c3 Y']:<8} {int(c3_calc):<8}"
            f"{states['/p3 Y']:<8} {states['/g3 Y']:<8}{states['/c4 Y']:<8} {int(c4_calc):<8}"
        )
        if states['/c1 Y'] != c1_calc:
            errors.append((t, 'c1', states['/c1 Y'], c1_calc))
        if states['/c2 Y'] != c2_calc:
            errors.append((t, 'c2', states['/c2 Y'], c2_calc))
        if states['/c3 Y'] != c3_calc:
            errors.append((t, 'c3', states['/c3 Y'], c3_calc))
        if states['/c4 Y'] != c4_calc:
            errors.append((t, 'c4', states['/c4 Y'], c4_calc))

    return errors

file_path = 'Simulations Data/carry_lookahead_comb.csv'
data = pd.read_csv(file_path)

start_time = 2.5e-9
end_time = data['/c1 X'].max()
time_step = 5e-9

midpoints = get_midpoints(data, start_time, end_time, time_step)
logic_errors = check_boolean_logic(data, midpoints)

violations_df = pd.DataFrame(logic_errors, columns=['Time', 'Signal', 'Observed', 'Expected'])

if not violations_df.empty:
    print("Violations detected in logic evaluation:")
    print(violations_df.to_string(index=False))
else:
    print("All logic conditions satisfied.")


Validating Boolean Logic and Printing States...
Time         /c0 Y    /p0 Y    /g0 Y   /c1 Obs  /c1 Exp  /p1 Y    /g1 Y    /c2 Obs  /c2 Exp /p2 Y    /g2 Y     /c3 Obs  /c3 Exp /p3 Y    /g3 Y     /c4 Obs  /c4 Exp 
--------------------------------------------------------------------------------------------------------------------------------------------
2.50e-09     0        0        0          0        0        0        0         0        0       0        0       0        0       0        0       0        0       
7.50e-09     0        0        0          0        0        0        0         0        0       0        0       0        0       0        0       0        0       
1.25e-08     0        0        0          0        0        0        0         0        0       0        0       0        0       0        0       0        0       
1.75e-08     1        1        0          1        1        0        0         0        0       0        1       1        1       0        0       0   

In [ ]:
def get_midpoints(data, start, end, step):
    return np.arange(start, end, step)

def check_boolean_logic(data, midpoints):
    errors = []

    for t in midpoints:
        # Identify rows before and after the midpoint
        before = data[data['/c1 X'] <= t].iloc[-1]
        after = data[data['/c1 X'] > t].iloc[0]

        # Linearly interpolate signal values at the midpoint
        interpolated = {}
        for col in data.columns:
            if ' X' not in col:  # Skip non-signal columns
                t1, t2 = before['/c1 X'], after['/c1 X']
                v1, v2 = before[col], after[col]
                interpolated[col] = v1 + (v2 - v1) * (t - t1) / (t2 - t1)

        # Evaluate signal states (>0.5 = 1, <=0.5 = 0)
        states = {key: int(interpolated[key] > 0.5) for key in interpolated.keys()}
        
        # Logic expectations
        c1_calc = states['/g0 Y'] or (states['/p0 Y'] and states['/c0 Y'])
        c2_calc = (
            states['/g1 Y'] or
            (states['/p1 Y'] and states['/g0 Y']) or
            (states['/p1 Y'] and states['/p0 Y'] and states['/c0 Y'])
        )
        c3_calc = (
            states['/g2 Y'] or
            (states['/p2 Y'] and states['/g1 Y']) or
            (states['/p2 Y'] and states['/p1 Y'] and states['/g0 Y']) or
            (states['/p2 Y'] and states['/p1 Y'] and states['/p0 Y'] and states['/c0 Y'])
        )
        c4_calc = (
            states['/g3 Y'] or
            (states['/p3 Y'] and states['/g2 Y']) or
            (states['/p3 Y'] and states['/p2 Y'] and states['/g1 Y']) or
            (states['/p3 Y'] and states['/p2 Y'] and states['/p1 Y'] and states['/g0 Y']) or
            (states['/p3 Y'] and states['/p2 Y'] and states['/p1 Y'] and states['/p0 Y'] and states['/c0 Y'])
        )

        # Compare actual and expected logic
        if states['/c1 Y'] != c1_calc:
            errors.append((t, 'c1', states['/c1 Y'], c1_calc))
        if states['/c2 Y'] != c2_calc:
            errors.append((t, 'c2', states['/c2 Y'], c2_calc))
        if states['/c3 Y'] != c3_calc:
            errors.append((t, 'c3', states['/c3 Y'], c3_calc))
        if states['/c4 Y'] != c4_calc:
            errors.append((t, 'c4', states['/c4 Y'], c4_calc))

    return errors

# Read input data
file_path = 'Simulations Data/carry_lookahead_comb.csv'
data = pd.read_csv(file_path)

# Define parameters for midpoint calculation
start_time = 2.5e-9
end_time = data['/c1 X'].max()
time_step = 5e-9

# Generate midpoints and evaluate logic
midpoints = get_midpoints(data, start_time, end_time, time_step)
logic_errors = check_boolean_logic(data, midpoints)

# Tabulate results for analysis
violations_df = pd.DataFrame(logic_errors, columns=['Time', 'Signal', 'Observed', 'Expected'])

# Display results
if not violations_df.empty:
    print("Violations detected in logic evaluation:")
    print(violations_df.to_string(index=False))
else:
    print("All logic conditions satisfied.")
